In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from pykrige.ok import OrdinaryKriging
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, colors

# ==============================================
# 【1. スクリプトと同じフォルダ内のCSVファイルをすべて取得】
# ==============================================
script_dir = os.path.dirname(__file__)
csv_files = glob.glob(os.path.join(script_dir, "*.csv"))
if not csv_files:
    raise FileNotFoundError("同じフォルダにCSVファイルが見つかりません。")

# ==============================================
# 【2. 物性値ごとのカラーバー表示範囲を設定】
# ==============================================
color_ranges = {
    "SiO2 Etching depth": (0, 1700),
    "SiO2 Etching rate": (0, 600),
    "Mask Etching depth": (0, 1400),
    "Mask Selectivity ratio": (1, 6),
    "Vpp": (0, 1000),
    "Vdc": (-800, 0)
}

# ==============================================
# 【3. 各CSVファイルについて処理を実施】
# ==============================================
for file_path in csv_files:
    print(f"読み込み中: {file_path}")

    # ==========================================
    # 【3-1. CSVファイルの読み込み（Shift-JIS対応）】
    # ==========================================
    df = pd.read_csv(file_path, encoding="shift_jis", header=None)

    # ==========================================
    # 【3-2. 物性値名（target_name）の取得と単位削除】
    # 1行目から対象値名を取得し、単位（[ ]内）を削除します。
    # ==========================================
    full_name = str(df.iloc[0,0])
    target_name = full_name.split("/", 1)[-1].strip()
    target_name = re.sub(r"\[.*?\]", "", target_name).strip()
    print(f"対象値: {target_name}")

    # ==========================================
    # 【3-3. Coil, Platen, 測定値(z)の抽出】
    # Coil（X軸）、Platen（Y軸）、測定値zを取得します。
    # ==========================================
    coil_x = df.iloc[1,1:].astype(float).values
    platen_y = df.iloc[3:,0].astype(float).values
    z_values = df.iloc[3:,1:].astype(float).values

    # ==========================================
    # 【3-4. x, y, z のリスト展開】
    # CoilとPlatenの全組み合わせについて測定値zを1次元配列化します。
    # ==========================================
    x_list, y_list, z_list = [], [], []
    for i, y_val in enumerate(platen_y):
        for j, x_val in enumerate(coil_x):
            x_list.append(x_val)
            y_list.append(y_val)
            z_list.append(z_values[i, j])
    x = np.array(x_list)
    y = np.array(y_list)
    z = np.array(z_list)

    # ==========================================
    # 【3-5. クリギング補間によるグリッド値の予測】
    # 測定点からグリッド状の値を空間補間で推定します。
    # ==========================================
    OK = OrdinaryKriging(x, y, z, variogram_model='linear', verbose=False, enable_plotting=False)
    xi = np.linspace(min(x), max(x), 100)
    yi = np.linspace(min(y), max(y), 50)
    Zi, ss = OK.execute("grid", xi, yi)
    Xi, Yi = np.meshgrid(xi, yi)

    # ==========================================
    # 【3-6. Vdc以外は下限値を0に制限（物理的に負にならないようクリップ）】
    # ==========================================
    if target_name != "Vdc":
        Zi = np.clip(Zi, 0, None)

    # ==========================================
    # 【3-7. カラーバー範囲（vmin, vmax）設定】
    # 物性値ごとに色の表示範囲を設定。設定がなければ自動。
    # ==========================================
    vmin, vmax = color_ranges.get(target_name, (None, None))
    norm = colors.Normalize(vmin=vmin, vmax=vmax)

    # ==========================================
    # 【3-8. 3Dサーフェス＆測定点のグラフ描画・保存】
    # クリギング補間値を3DサーフェスとしてプロットしPNG保存します。
    # ==========================================
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(Xi, Yi, Zi, cmap=cm.viridis, edgecolor='none', alpha=0.9, norm=norm)
    ax.scatter(x, y, z, c='red', s=50, edgecolors="black", label="Measured")
    ax.set_xlabel("Coil [W]")
    ax.set_ylabel("Platen [W]")
    ax.set_zlabel(target_name)
    ax.set_title(f"3D Mapping of {target_name}")
    mappable = cm.ScalarMappable(norm=norm, cmap=cm.viridis)
    mappable.set_array(Zi)
    fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10, label=target_name)
    plt.legend()

    safe_name = re.sub(r'[\\/:*?"<>|]', '_', str(target_name))
    output_file_3d = f"{safe_name}_3D_surface.png"
    plt.savefig(os.path.join(script_dir, output_file_3d), dpi=300, bbox_inches="tight")
    plt.close()
    print(f"3Dサーフェスのグラフを保存しました: {output_file_3d}")

    # ==========================================
    # 【3-9. 2D等高線グラフの描画・保存】
    # クリギング補間値を2D等高線としてプロットしPNG保存します。
    # ==========================================
    fig2 = plt.figure(figsize=(10,8))
    cp = plt.contourf(Xi, Yi, Zi, cmap=cm.viridis, levels=np.linspace(vmin, vmax, 21), norm=norm)
    plt.scatter(x, y, c='red', s=40, edgecolors="black", label="Measured")
    plt.xlabel("Coil [W]")
    plt.ylabel("Platen [W]")
    plt.title(f"2D Contour of {target_name}")
    plt.colorbar(cp, label=target_name)
    plt.legend()

    output_file_2d = f"{safe_name}_2D_contour.png"
    plt.savefig(os.path.join(script_dir, output_file_2d), dpi=300, bbox_inches="tight")
    plt.close()
    print(f"2D等高線のグラフを保存しました: {output_file_2d}")